In [4]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt
import os
from os.path import join
from collections import defaultdict
import sys
import joblib
import auggam.config
import dvu
dvu.set_style()
sys.path.append('../results')
pd.set_option('display.max_rows', None)
RESULTS_DIR = join(auggam.config.results_dir, '7gram')

fnames = [fname for fname in os.listdir(RESULTS_DIR)] # if fname.startswith('instructor')]
# accs = pd.read_pickle('../results/instructor_accs.pkl')
fnames_acc = [fname for fname in fnames if 'acc_' in fname]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
df = defaultdict(list)
def rename_bert_models(s):
    if 'bert' in s.lower() and not s == 'bert-base-uncased':
        return 'bert-finetuned'
    else:
        return s

for fname in fnames_acc:
    # print(fname)
    acc = joblib.load(join(RESULTS_DIR, fname))
    # print(acc)
    df['acc'].append(acc['acc_val'])
    # print(fname.split('_acc_'))
    df['model'].append(fname.split('_acc_')[0])
    df['dset'].append(fname.split('_acc_')[1].replace('_imodelsx.pkl', ''))
# joblib.load(join('../results', fnames_acc[0]))
df = pd.DataFrame(df)
df['model'] = df['model'].apply(rename_bert_models)
# df

In [6]:
d = (
    df.pivot(index='model', columns='dset', values='acc').round(3)
)
d.style.format(precision=3).background_gradient(cmap='Blues', axis=0)

dset,ag_news,dbpedia_14,emotion,financial_phrasebank,rotten_tomatoes,sst2,trec
model,,,,,,,
bert-base-uncased,0.888,nan,0.654,0.854,0.781,0.820,0.890
bert-finetuned,0.906,nan,0.898,0.922,0.815,0.877,0.944
gpt2,0.861,0.911,0.496,0.719,0.762,0.714,0.796
gpt2-xl,0.864,nan,0.680,0.760,0.815,0.788,0.902
instructor,nan,nan,0.710,0.761,0.800,0.847,nan
linear_finetune,0.904,0.984,0.648,0.863,0.813,0.853,0.928
llama_7b,0.868,nan,0.734,0.821,0.820,0.817,0.932
tfidfvectorizer,0.879,0.957,0.860,0.833,0.750,0.811,0.874
